In [62]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd

In [64]:
!which chromedriver

/usr/local/bin/chromedriver


In [65]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

browser.visit(url)


In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_article = soup.find_all('div', class_='content_title')[1].a.text
news_p=soup.find('div', class_='article_teaser_body').text


In [6]:
url2="https://www.jpl.nasa.gov/spaceimages/"
browser.visit(url2)

In [7]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')



In [8]:
img_url = soup.find('article', class_='carousel_item')["style"].split("'")[1]
img_url

'/spaceimages/images/wallpaper/PIA18904-1920x1200.jpg'

In [9]:

featured_image_url="http://www.jpl.nasa.gov"+img_url



featured_image_url


'http://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18904-1920x1200.jpg'

In [69]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

html = browser.html
weather_soup = BeautifulSoup(html, "html.parser")
tweet_attrs = {"class": "tweet", "data-name": "Mars Weather"}
mars_weather_tweet = weather_soup.find("div", attrs=tweet_attrs)
mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()
    # First, find a tweet with the data-name `Mars Weather`


 

AttributeError: 'NoneType' object has no attribute 'find'

In [67]:

    # Next, search within the tweet for the p tag containing the tweet text


In [68]:
mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()

AttributeError: 'NoneType' object has no attribute 'find'

In [20]:
astro_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
head_url=astro_url.split("search")[0]
astro=requests.get(astro_url)

In [22]:
soup_ast=BeautifulSoup(astro.text, "html.parser")

In [27]:
hemp_list=soup_ast.find_all('div', class_="item")

In [39]:
hemp_list[0].img["src"]

'/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'

In [25]:
hemisphere_images_url =[]

In [43]:
hemp_list=soup_ast.find_all('div', class_="item")

# Loop through returned results
for hemp in hemp_list:
    # Error handling
    try:
        # Identify and return title of listing
         title= hemp.find('h3').text
        # Identify and return price of listing
         hem_imp= hemp.img["src"]
        # Identify and return link to listing
         image=head_url+img_url

        # Run only if title, price, and link are available
         if (title and image):
            # Print results
            print('-------------')
            print(title)
            print(image)
           


            # Dictionary to be inserted as a MongoDB document
            post = {
                'titles': title,
                'image_url': image
            }

            hemisphere_images_url.append(post)
    except Exception as e:
        print(e)

-------------
Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg
-------------
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg
-------------
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg
-------------
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov//spaceimages/images/wallpaper/PIA18904-1920x1200.jpg


In [44]:
facts_url ="https://space-facts.com/mars/"

In [48]:
tables = pd.read_html(facts_url)
Mars_df = tables[2]

In [50]:
Mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [75]:
d=[]
dataz={0: "description",1:""}
d.append(dataz)


In [77]:

Mars_df=pd.concat([pd.DataFrame(d), Mars_df], ignore_index=True)

In [79]:
Mars_df.columns=['', 'value']

In [80]:
Mars_df.head()

,,value
0,description,
1,Equatorial Diameter:,"6,792 km"
2,Polar Diameter:,"6,752 km"
3,Mass:,6.39 × 10^23 kg (0.11 Earths)
4,Moons:,2 (Phobos & Deimos)


In [84]:
html_table = Mars_df.to_html(index=False)
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>description</td>\n      <td></td>\n    </tr>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [85]:
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>  </thead>  <tbody>    <tr>      <td>description</td>      <td></td>    </tr>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [86]:
Mars_df.to_html('Mars.html')